Import packages

In [2]:
#https://spark.apache.org/docs/latest/api/python/pyspark.sql.html
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

Read epa data in from Azure Storage, cluster must be configured to access WASB

In [4]:
epaMpg = spark.read.csv('wasb://spark@dbshepstor.blob.core.windows.net/data/epaMpg.csv', header=True, inferSchema=True)
epaMpg.show()

+---+-------------------------+--------------------+------------+----------+---------+-----------------------------+------------------------+-----+-----------------+------+---------+-----------------+--------------------------+-----------------+--------------------------+--------+
_c0|Represented.Test.Veh.Make| Model|Vehicle.Type|HorsePower|Cylinders|Tested.Transmission.Type.Code|Tested.Transmission.Type|Gears|Drive.System.Code|Weight|AxleRatio|Test.Procedure.Cd|Test.Procedure.Description|Test.Fuel.Type.Cd|Test.Fuel.Type.Description|FuelEcon|
+---+-------------------------+--------------------+------------+----------+---------+-----------------------------+------------------------+-----+-----------------+------+---------+-----------------+--------------------------+-----------------+--------------------------+--------+
 1| Aston Martin| Rapide S| Car| 552| 12| SA| Semi-Automatic| 8| R| 4750| 2.73| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 17.3|
 2| Aston Martin| Vanquish| Car| 568| 12| SA| Semi-Automatic| 8| R| 4500| 2.73| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 16.5|
 3| BENTLEY| Continental GT| Car| 616| 12| SA| Semi-Automatic| 8| F| 6000| 2.85| 90| US06| 61| Tier 2 Cert Gasoline| 17.4|
 4| BENTLEY| Continental GT| Car| 616| 12| SA| Semi-Automatic| 8| F| 6000| 2.85| 11| Cold CO| 27| Cold CO Premium (...| 13.6|
 5| BMW| 230i Convertible| Car| 248| 4| SA| Semi-Automatic| 8| R| 4000| 2.81| 3| HWFE| 61| Tier 2 Cert Gasoline| 45.8|
 6| BMW| 230i Coupe| Car| 248| 4| M| Manual| 6| R| 3625| 3.91| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 26.4|
 7| BMW| 230i Coupe| Car| 248| 4| SA| Semi-Automatic| 8| R| 3625| 2.81| 3| HWFE| 61| Tier 2 Cert Gasoline| 50.6|
 8| BMW|230i xDrive Conve...| Car| 248| 4| SA| Semi-Automatic| 8| R| 4000| 2.81| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.6|
 9| BMW| 230i xDrive Coupe| Car| 248| 4| SA| Semi-Automatic| 8| R| 3750| 2.81| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 30.3|
 10| BMW| 320i| Both| 181| 4| A| Automatic| 8| R| 3625| 3.2| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 30.4|
 11| BMW| 320i| Both| 180| 4| M| Manual| 6| R| 3625| 3.08| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.5|
 12| BMW| 320i xDrive| Both| 181| 4| A| Automatic| 8| R| 3750| 3.15| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.9|
 13| BMW| 328d| Car| 183| 4| SA| Semi-Automatic| 8| R| 3750| 2.81| 2| CVS 75 and later ...| 19| Federal Cert Dies...| 42.2|
 14| BMW| 328d xDrive| Car| 181| 4| SA| Semi-Automatic| 8| R| 4000| 2.81| 2| CVS 75 and later ...| 19| Federal Cert Dies...| 41.2|
 15| BMW|328d xDrive Sport...| Car| 181| 4| SA| Semi-Automatic| 8| R| 4250| 2.81| 3| HWFE| 19| Federal Cert Dies...| 57.2|
 16| BMW| 330e| Car| 180| 4| SA| Semi-Automatic| 8| R| 4250| 2.93| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 35.6|
 17| BMW| 330i| Car| 248| 4| SA| Semi-Automatic| 8| R| 3875| 2.81| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 30.8|
 18| BMW| 330i| Car| 248| 4| M| Manual| 6| R| 3750| 3.91| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 26.7|
 19| BMW| 330i xDrive| Car| 248| 4| SA| Semi-Automatic| 8| R| 4000| 2.81| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.7|
 20| BMW|330i xDrive Gran ...| Car| 248| 4| SA| Semi-Automatic| 8| R| 4250| 2.93| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.3|
+---+-------------------------+--------------------+------------+----------+---------+-----------------------------+------------------------+-----+-----------------+------+---------+-----------------+--------------------------+-----------------+--------------------------+--------+
only showing top 20 rows

In [5]:
type(epaMpg)

Out[ 3 ]: pyspark.sql.dataframe.DataFrame

In [6]:
epaMpg.printSchema()

root
-- _c0: integer (nullable = true)
-- Represented.Test.Veh.Make: string (nullable = true)
-- Model: string (nullable = true)
-- Vehicle.Type: string (nullable = true)
-- HorsePower: integer (nullable = true)
-- Cylinders: integer (nullable = true)
-- Tested.Transmission.Type.Code: string (nullable = true)
-- Tested.Transmission.Type: string (nullable = true)
-- Gears: integer (nullable = true)
-- Drive.System.Code: string (nullable = true)
-- Weight: integer (nullable = true)
-- AxleRatio: double (nullable = true)
-- Test.Procedure.Cd: integer (nullable = true)
-- Test.Procedure.Description: string (nullable = true)
-- Test.Fuel.Type.Cd: integer (nullable = true)
-- Test.Fuel.Type.Description: string (nullable = true)
-- FuelEcon: double (nullable = true)

In [7]:
epaMpg.columns

Out[ 5 ]: 
['_c0',
 'Represented.Test.Veh.Make',
 'Model',
 'Vehicle.Type',
 'HorsePower',
 'Cylinders',
 'Tested.Transmission.Type.Code',
 'Tested.Transmission.Type',
 'Gears',
 'Drive.System.Code',
 'Weight',
 'AxleRatio',
 'Test.Procedure.Cd',
 'Test.Procedure.Description',
 'Test.Fuel.Type.Cd',
 'Test.Fuel.Type.Description',
 'FuelEcon']

In [8]:
epaMpg.corr('FuelEcon','Weight')

Out[ 6 ]: -0.5713160000548484

In [9]:
epaMpg.select("`Represented.Test.Veh.Make`").distinct().count()

Out[ 7 ]: 45

In [10]:
epaMpg.filter(epaMpg.FuelEcon > 50).count()

Out[ 8 ]: 30

In [11]:
epaMpg.first()

Out[ 9 ]: Row(_c0=1, Represented.Test.Veh.Make='Aston Martin', Model='Rapide S', Vehicle.Type='Car', HorsePower=552, Cylinders=12, Tested.Transmission.Type.Code='SA', Tested.Transmission.Type='Semi-Automatic', Gears=8, Drive.System.Code='R', Weight=4750, AxleRatio=2.73, Test.Procedure.Cd=21, Test.Procedure.Description='Federal fuel 2-day exhaust (w/can load)', Test.Fuel.Type.Cd=61, Test.Fuel.Type.Description='Tier 2 Cert Gasoline', FuelEcon=17.3)

In [12]:

epaMpg.groupBy("`Represented.Test.Veh.Make`").agg({'FuelEcon': 'mean'}).collect()

Out[ 10 ]: 
[Row(Represented.Test.Veh.Make='ACURA', avg(FuelEcon)=32.275),
 Row(Represented.Test.Veh.Make='LAMBORGHINI', avg(FuelEcon)=12.87),
 Row(Represented.Test.Veh.Make='Jaguar', avg(FuelEcon)=29.876470588235296),
 Row(Represented.Test.Veh.Make='HYUNDAI', avg(FuelEcon)=34.52142857142857),
 Row(Represented.Test.Veh.Make='Jeep', avg(FuelEcon)=24.722727272727273),
 Row(Represented.Test.Veh.Make='Mini', avg(FuelEcon)=34.30857142857142),
 Row(Represented.Test.Veh.Make='Kia', avg(FuelEcon)=31.15757575757576),
 Row(Represented.Test.Veh.Make='TOYOTA', avg(FuelEcon)=33.68695652173914),
 Row(Represented.Test.Veh.Make='RAM', avg(FuelEcon)=21.560000000000002),
 Row(Represented.Test.Veh.Make='Volvo', avg(FuelEcon)=32.47368421052631),
 Row(Represented.Test.Veh.Make='SUBARU', avg(FuelEcon)=30.95),
 Row(Represented.Test.Veh.Make='NISSAN', avg(FuelEcon)=25.27391304347826),
 Row(Represented.Test.Veh.Make='Hyundai', avg(FuelEcon)=30.55588235294118),
 Row(Represented.Test.Veh.Make='BENTLEY', avg(FuelEcon)=14.533333333333335),
 Row(Represented.Test.Veh.Make='INFINITI', avg(FuelEcon)=29.292857142857137),
 Row(Represented.Test.Veh.Make='FORD', avg(FuelEcon)=19.2),
 Row(Represented.Test.Veh.Make='AUDI', avg(FuelEcon)=24.41891891891892),
 Row(Represented.Test.Veh.Make='MINI', avg(FuelEcon)=26.55),
 Row(Represented.Test.Veh.Make='Karma', avg(FuelEcon)=19.2),
 Row(Represented.Test.Veh.Make='CHEVROLET', avg(FuelEcon)=28.38461538461538),
 Row(Represented.Test.Veh.Make='BUGATTI', avg(FuelEcon)=9.8),
 Row(Represented.Test.Veh.Make='Land Rover', avg(FuelEcon)=27.150000000000002),
 Row(Represented.Test.Veh.Make='Alfa Romeo', avg(FuelEcon)=27.599999999999998),
 Row(Represented.Test.Veh.Make='Mercedes-Benz', avg(FuelEcon)=25.06875),
 Row(Represented.Test.Veh.Make='LEXUS', avg(FuelEcon)=30.977272727272727),
 Row(Represented.Test.Veh.Make='Rolls Royce', avg(FuelEcon)=14.479999999999999),
 Row(Represented.Test.Veh.Make='MAZDA', avg(FuelEcon)=40.554545454545455),
 Row(Represented.Test.Veh.Make='Chrysler', avg(FuelEcon)=24.357142857142858),
 Row(Represented.Test.Veh.Make='Lincoln', avg(FuelEcon)=24.14),
 Row(Represented.Test.Veh.Make='BMW', avg(FuelEcon)=28.123529411764718),
 Row(Represented.Test.Veh.Make='MASERATI', avg(FuelEcon)=18.6),
 Row(Represented.Test.Veh.Make='VOLKSWAGEN', avg(FuelEcon)=28.6516129032258),
 Row(Represented.Test.Veh.Make='KIA', avg(FuelEcon)=41.41875),
 Row(Represented.Test.Veh.Make='Ferrari', avg(FuelEcon)=17.139999999999997),
 Row(Represented.Test.Veh.Make='HONDA', avg(FuelEcon)=33.3811320754717),
 Row(Represented.Test.Veh.Make='GMC', avg(FuelEcon)=24.3),
 Row(Represented.Test.Veh.Make='SCION', avg(FuelEcon)=43.7),
 Row(Represented.Test.Veh.Make='Porsche', avg(FuelEcon)=24.737499999999997),
 Row(Represented.Test.Veh.Make='CADILLAC', avg(FuelEcon)=24.20416666666667),
 Row(Represented.Test.Veh.Make='Aston Martin', avg(FuelEcon)=16.9),
 Row(Represented.Test.Veh.Make='Dodge', avg(FuelEcon)=19.358823529411765),
 Row(Represented.Test.Veh.Make='Fiat', avg(FuelEcon)=30.160000000000004),
 Row(Represented.Test.Veh.Make='Ford', avg(FuelEcon)=28.94393939393939),
 Row(Represented.Test.Veh.Make='BUICK', avg(FuelEcon)=31.18),
 Row(Represented.Test.Veh.Make='MITSUBISHI', avg(FuelEcon)=48.940000000000005)]

In [13]:
%sql 
select `Represented.Test.Veh.Make`, avg(FuelEcon) as AvgEcon
from epaMpg
group by `Represented.Test.Veh.Make`
order by AvgEcon

Represented.Test.Veh.Make,AvgEcon
BUGATTI,9.8
LAMBORGHINI,12.87
Rolls Royce,14.479999999999999
BENTLEY,14.533333333333335
Aston Martin,16.9
Ferrari,17.139999999999997
MASERATI,18.6
FORD,19.2
Karma,19.2
Dodge,19.358823529411765


In [14]:
epaMpg.count()

Out[ 11 ]: 1034

In [15]:
epaMpg.count() - epaMpg.dropDuplicates().count()

Out[ 12 ]: 0

In [16]:

carModels = epaMpg.select("`Model`", "`Represented.Test.Veh.Make`")
carModels.show()



+--------------------+-------------------------+
 Model|Represented.Test.Veh.Make|
+--------------------+-------------------------+
 Rapide S| Aston Martin|
 Vanquish| Aston Martin|
 Continental GT| BENTLEY|
 Continental GT| BENTLEY|
 230i Convertible| BMW|
 230i Coupe| BMW|
 230i Coupe| BMW|
230i xDrive Conve...| BMW|
 230i xDrive Coupe| BMW|
 320i| BMW|
 320i| BMW|
 320i xDrive| BMW|
 328d| BMW|
 328d xDrive| BMW|
328d xDrive Sport...| BMW|
 330e| BMW|
 330i| BMW|
 330i| BMW|
 330i xDrive| BMW|
330i xDrive Gran ...| BMW|
+--------------------+-------------------------+
only showing top 20 rows

Register a dataframe as a globalb temp table

In [18]:


carModels.createGlobalTempView("globalCarModels")




In [19]:

spark.sql("SELECT * FROM global_temp.globalCarModels").show()


+--------------------+-------------------------+
 Model|Represented.Test.Veh.Make|
+--------------------+-------------------------+
 Rapide S| Aston Martin|
 Vanquish| Aston Martin|
 Continental GT| BENTLEY|
 Continental GT| BENTLEY|
 230i Convertible| BMW|
 230i Coupe| BMW|
 230i Coupe| BMW|
230i xDrive Conve...| BMW|
 230i xDrive Coupe| BMW|
 320i| BMW|
 320i| BMW|
 320i xDrive| BMW|
 328d| BMW|
 328d xDrive| BMW|
328d xDrive Sport...| BMW|
 330e| BMW|
 330i| BMW|
 330i| BMW|
 330i xDrive| BMW|
330i xDrive Gran ...| BMW|
+--------------------+-------------------------+
only showing top 20 rows

In [20]:
#epaMpg.describe().show()

epaMpg.select("FuelEcon", "HorsePower", "Weight", "Gears","`Tested.Transmission.Type`").describe().show()


+-------+------------------+------------------+-----------------+------------------+------------------------+
summary| FuelEcon| HorsePower| Weight| Gears|Tested.Transmission.Type|
+-------+------------------+------------------+-----------------+------------------+------------------------+
 count| 1034| 1034| 1034| 1034| 1034|
 mean|28.216537717601547|291.82495164410057|4191.852030947775| 6.509671179883946| null|
 stddev| 9.496232870303718| 144.2949320373475|787.8214341838323|1.9928243220390303| null|
 min| 9.2| 72| 2375| 1| Automated Manual|
 max| 71.6| 1500| 6500| 10| Semi-Automatic|
+-------+------------------+------------------+-----------------+------------------+------------------------+

In [21]:
spark.sql("select * from epaMpg limit 10").show()

+---+-------------------------+--------------------+------------+----------+---------+-----------------------------+------------------------+-----+-----------------+------+---------+-----------------+--------------------------+-----------------+--------------------------+--------+
_c0|Represented.Test.Veh.Make| Model|Vehicle.Type|HorsePower|Cylinders|Tested.Transmission.Type.Code|Tested.Transmission.Type|Gears|Drive.System.Code|Weight|AxleRatio|Test.Procedure.Cd|Test.Procedure.Description|Test.Fuel.Type.Cd|Test.Fuel.Type.Description|FuelEcon|
+---+-------------------------+--------------------+------------+----------+---------+-----------------------------+------------------------+-----+-----------------+------+---------+-----------------+--------------------------+-----------------+--------------------------+--------+
 1| Aston Martin| Rapide S| Car| 552| 12| SA| Semi-Automatic| 8| R| 4750| 2.73| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 17.3|
 2| Aston Martin| Vanquish| Car| 568| 12| SA| Semi-Automatic| 8| R| 4500| 2.73| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 16.5|
 3| BENTLEY| Continental GT| Car| 616| 12| SA| Semi-Automatic| 8| F| 6000| 2.85| 90| US06| 61| Tier 2 Cert Gasoline| 17.4|
 4| BENTLEY| Continental GT| Car| 616| 12| SA| Semi-Automatic| 8| F| 6000| 2.85| 11| Cold CO| 27| Cold CO Premium (...| 13.6|
 5| BMW| 230i Convertible| Car| 248| 4| SA| Semi-Automatic| 8| R| 4000| 2.81| 3| HWFE| 61| Tier 2 Cert Gasoline| 45.8|
 6| BMW| 230i Coupe| Car| 248| 4| M| Manual| 6| R| 3625| 3.91| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 26.4|
 7| BMW| 230i Coupe| Car| 248| 4| SA| Semi-Automatic| 8| R| 3625| 2.81| 3| HWFE| 61| Tier 2 Cert Gasoline| 50.6|
 8| BMW|230i xDrive Conve...| Car| 248| 4| SA| Semi-Automatic| 8| R| 4000| 2.81| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.6|
 9| BMW| 230i xDrive Coupe| Car| 248| 4| SA| Semi-Automatic| 8| R| 3750| 2.81| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 30.3|
 10| BMW| 320i| Both| 181| 4| A| Automatic| 8| R| 3625| 3.2| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 30.4|
+---+-------------------------+--------------------+------------+----------+---------+-----------------------------+------------------------+-----+-----------------+------+---------+-----------------+--------------------------+-----------------+--------------------------+--------+

In [22]:
%sql 
select `Represented.Test.Veh.Make`,Model,HorsePower,FuelEcon
from epaMpg

order by HorsePower 

Represented.Test.Veh.Make,Model,HorsePower,FuelEcon
TOYOTA,PRIUS c,72,52.3
TOYOTA,PRIUS c,72,51.7
TOYOTA,PRIUS c,72,41.3
TOYOTA,PRIUS c,72,43.7
MITSUBISHI,MIRAGE G4,78,45.1
MITSUBISHI,MIRAGE,78,44.7
MITSUBISHI,MIRAGE G4,78,43.7
MITSUBISHI,MIRAGE,78,46.5
MITSUBISHI,MIRAGE,78,64.7
CHEVROLET,SPARK,84,38.9


In [23]:
%sql 
select HorsePower,FuelEcon from epaMpg

HorsePower,FuelEcon
552,17.3
568,16.5
616,17.4
616,13.6
248,45.8
248,26.4
248,50.6
248,29.6
248,30.3
181,30.4


In [24]:
epaMpg.corr("HorsePower","FuelEcon")

Out[ 18 ]: -0.6295576916726039

In [25]:
%sql 
select Fuelecon,Weight from epaMpg

Fuelecon,Weight
17.3,4750
16.5,4500
17.4,6000
13.6,6000
45.8,4000
26.4,3625
50.6,3625
29.6,4000
30.3,3750
30.4,3625


In [26]:
%sql 
select HorsePower,Weight from epaMpg

HorsePower,Weight
552,4750
568,4500
616,6000
616,6000
248,4000
248,3625
248,3625
248,4000
248,3750
181,3625


In [27]:
%sql 
select `Represented.Test.Veh.Make`,Model,HorsePower,FuelEcon
from epaMpg
WHERE HorsePower > 400 and FuelEcon > 30
order by FuelEcon 

Represented.Test.Veh.Make,Model,HorsePower,FuelEcon
Mercedes-Benz,AMG GT S,503,30.2
AUDI,R8,540,31.5
CHEVROLET,CORVETTE,455,31.7
CHEVROLET,CORVETTE,455,31.7
Hyundai,G90,420,32.1
CADILLAC,XTS AWD,410,32.2
CHEVROLET,CORVETTE,455,33.3
Mercedes-Benz,SL 550,449,34.5
BMW,M550i xDrive,455,35.2
Mercedes-Benz,S 560 4MATIC Maybach,463,35.2


In [28]:
epaData = epaMpg.select("FuelEcon", "HorsePower", "Weight", "Gears")

In [29]:


splits = epaData.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 703 Testing Rows: 331

In [30]:
train.show()

+--------+----------+------+-----+
FuelEcon|HorsePower|Weight|Gears|
+--------+----------+------+-----+
 10.4| 1500| 4750| 7|
 11.0| 700| 6000| 8|
 11.2| 700| 4250| 7|
 11.4| 381| 6000| 6|
 11.8| 740| 4250| 7|
 12.0| 700| 4250| 7|
 12.5| 700| 6000| 8|
 12.6| 700| 4000| 7|
 12.8| 520| 6000| 8|
 12.9| 560| 4750| 8|
 13.1| 345| 6500| 6|
 13.3| 261| 4750| 5|
 13.4| 261| 4750| 5|
 13.5| 610| 4250| 7|
 13.5| 630| 4000| 7|
 13.6| 616| 6000| 8|
 13.7| 707| 4750| 6|
 13.7| 707| 6000| 8|
 13.9| 261| 4500| 5|
 13.9| 630| 3750| 7|
+--------+----------+------+-----+
only showing top 20 rows

In [31]:
#"FuelEcon", "HorsePower", "Weight", "Gears","`Tested.Transmission.Type`"
# Now we'll assemble a vector of all the numeric feature columns (other than ArrDelay, which we wouldn't have for enroute flights)
assembler = VectorAssembler(inputCols = ["FuelEcon", "HorsePower", "Weight", "Gears"], outputCol="features")

type(assembler)

Out[ 22 ]: pyspark.ml.feature.VectorAssembler

In [32]:
training = assembler.transform(train).select(col("features"), (col("FuelEcon").cast("Int").alias("label")))
training.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
[10.4,1500.0,4750...| 10|
[11.0,700.0,6000....| 11|
[11.2,700.0,4250....| 11|
[11.4,381.0,6000....| 11|
[11.8,740.0,4250....| 11|
[12.0,700.0,4250....| 12|
[12.5,700.0,6000....| 12|
[12.6,700.0,4000....| 12|
[12.8,520.0,6000....| 12|
[12.9,560.0,4750....| 12|
[13.1,345.0,6500....| 13|
[13.3,261.0,4750....| 13|
[13.4,261.0,4750....| 13|
[13.5,610.0,4250....| 13|
[13.5,630.0,4000....| 13|
[13.6,616.0,6000....| 13|
[13.7,707.0,4750....| 13|
[13.7,707.0,6000....| 13|
[13.9,261.0,4500....| 13|
[13.9,630.0,3750....| 13|
+--------------------+-----+
only showing top 20 rows

In [33]:
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)
model = lr.fit(training)
print ("Model trained!")

Model trained!

In [34]:
# Generate the features vector and label
testing = assembler.transform(test).select(col("features"), (col("FuelEcon")).cast("Int").alias("trueLabel"))
testing.show()

+--------------------+---------+
 features|trueLabel|
+--------------------+---------+
[9.2,1500.0,4750....| 9|
[10.0,740.0,4250....| 10|
[11.3,381.0,6500....| 11|
[11.7,345.0,6500....| 11|
[11.9,740.0,4000....| 11|
[12.7,275.0,6000....| 12|
[13.5,285.0,6000....| 13|
[13.5,621.0,4500....| 13|
[13.8,530.0,6500....| 13|
[13.8,680.0,4750....| 13|
[14.0,345.0,6000....| 14|
[14.6,563.0,6000....| 14|
[14.6,785.0,4250....| 14|
[14.8,600.0,6000....| 14|
[15.0,395.0,5250....| 15|
[15.0,605.0,5000....| 15|
[15.1,540.0,4250....| 15|
[15.3,650.0,4500....| 15|
[15.7,470.0,6000....| 15|
[15.7,600.0,6000....| 15|
+--------------------+---------+
only showing top 20 rows

In [35]:
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show()

+--------------------+------------------+---------+
 features| prediction|trueLabel|
+--------------------+------------------+---------+
[9.2,1500.0,4750....|7.8529949872510185| 9|
[10.0,740.0,4250....| 9.841305397883275| 10|
[11.3,381.0,6500....|11.150220724897766| 11|
[11.7,345.0,6500....|11.582047604596692| 11|
[11.9,740.0,4000....|11.689313594287716| 11|
[12.7,275.0,6000....|12.733897769816027| 12|
[13.5,285.0,6000....|13.475666441372757| 13|
[13.5,621.0,4500....|13.277449989149819| 13|
[13.8,530.0,6500....| 13.28358784756396| 13|
[13.8,680.0,4750....| 13.42248392895328| 13|
[14.0,345.0,6000....|13.859377672419368| 14|
[14.6,563.0,6000....|14.093180274561933| 14|
[14.6,785.0,4250....|14.125055303212703| 14|
[14.8,600.0,6000....| 14.22734152134723| 14|
[15.0,395.0,5250....|14.864593009264855| 15|
[15.0,605.0,5000....|14.613089752815098| 15|
[15.1,540.0,4250....|14.962119604887883| 15|
[15.3,650.0,4500....|14.922313933412154| 15|
[15.7,470.0,6000....|15.271785772672438| 15|
[15.7,600.0,6000....| 15.07855331633895| 15|
+--------------------+------------------+---------+
only showing top 20 rows